In [1]:
import os
import random
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [2]:

base_path = '/kaggle/input/fruits-dataset-for-classification'


fruit_label_mapping = {
    'fresh_peaches_done': 'fresh_peach',
    'fresh_pomegranates_done': 'fresh_pomegranate',
    'fresh_strawberries_done': 'fresh_strawberry',
    'rotten_peaches_done': 'rotten_peach',
    'rotten_pomegranates_done': 'rotten_pomegranate',
    'rotten_strawberries_done': 'rotten_strawberry'
}


def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB').resize((100, 100))
    return np.array(image) / 255.0  


all_images_data, all_labels = [], []
for dir_name, fruit_type in fruit_label_mapping.items():
    directory = os.path.join(base_path, dir_name)
    image_list = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('.jpg', '.jpeg', '.png'))]
    
    if image_list: 
        data = [load_and_preprocess_image(img) for img in image_list]
        all_images_data.extend(data)
        all_labels.extend([fruit_type] * len(data))


all_images_data, all_labels = np.array(all_images_data), np.array(all_labels)

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(all_labels)

train_images, val_images, train_labels, val_labels = train_test_split(
    all_images_data, encoded_labels, test_size=0.2, random_state=42
)



In [3]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
for layer in base_model.layers:
    layer.trainable = False  

x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
output = Dense(len(label_encoder.classes_), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=20, batch_size=32)
test_loss, test_acc = model.evaluate(val_images, val_labels)
print(f"Validation Accuracy: {test_acc:.4f}")


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 12s 154ms/step - accuracy: 0.2001 - loss: 2.1177 - val_accuracy: 0.4441 - val_loss: 1.5324
Epoch 2/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.3389 - loss: 1.6102 - val_accuracy: 0.6526 - val_loss: 1.2680
Epoch 3/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4461 - loss: 1.3981 - val_accuracy: 0.6677 - val_loss: 1.0928
Epoch 4/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.5467 - loss: 1.2199 - val_accuracy: 0.7009 - val_loss: 0.9884
Epoch 5/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.5553 - loss: 1.1496 - val_accuracy: 0.7160 - val_loss: 0.8880
Epoch 6/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.5955 - loss: 1.0647 - val_accuracy: 0.7281 - val_loss: 0.8292
Epoch 7/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6439 - loss: 0.9911 - val_accuracy: 0.7251 - val_loss: 0.8011
Epoch 8/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accur